In [1]:
import pandas as pd 
import numpy  as np  
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram    
import sys 
sys.setrecursionlimit(10000)  #increasing maximum recursion from 999 to 10K

from scipy.cluster.hierarchy import dendrogram, fcluster, leaves_list
from scipy.spatial import distance
from fastcluster import linkage
from sklearn.metrics.pairwise import pairwise_distances  



# needed imports
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage , fcluster


In [2]:
np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation 

In [3]:
pmpm=pd.read_csv("../Data/pmpm.csv",sep=",") #importing pmpm file.

In [4]:
pmpm.head()  

,new_euid,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,...,CHCC_G16,CHCC_G17,CHCC_G18,pcal,CHCC_SCORE_ADULT,imp_prov_npi,lob,med_prdt_dtl_mkt_seg_cd,rx_prdt_dtl_mkt_seg_cd,GENDER
0,718157.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1.160,1.702,1.962593e+09,2,COMCL,COMCL,F
1,718158.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1.473,0.726,1.184692e+09,2,COMCL,COMCL,F
2,718159.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.861,0.580,1.376855e+09,2,COMCL,COMCL,M
3,718161.0,0,0,0,0,0,0,0,0,0,...,0,0,0,2.592,7.361,1.912096e+09,2,COMCL,COMCL,M
4,718162.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1.777,0.726,1.871752e+09,2,COMCL,COMCL,F


In [5]:
pmpm_diabetic=pmpm[pmpm['CHCC_G01']==1] #selecting only diabetic group.  

In [6]:
pmpm_diabetic.shape   #so we are working on a subset of 34K diabetic patients

(34584, 152)

In [7]:
## Subsetting data with only codes  CHCCXXX codes
pmpm_diabetic=pmpm_diabetic[['new_euid','CHCC001','CHCC002','CHCC003','CHCC004','CHCC006','CHCC008','CHCC009','CHCC010','CHCC011','CHCC012','CHCC013','CHCC018','CHCC019','CHCC020','CHCC021','CHCC023','CHCC026','CHCC027','CHCC028','CHCC029','CHCC030','CHCC034','CHCC035','CHCC036','CHCC037','CHCC038','CHCC041','CHCC042','CHCC045','CHCC046','CHCC047','CHCC048','CHCC054','CHCC055','CHCC056','CHCC057','CHCC061','CHCC062','CHCC063','CHCC064','CHCC066','CHCC067','CHCC068','CHCC069','CHCC070','CHCC071','CHCC073','CHCC074','CHCC075','CHCC081','CHCC082','CHCC087','CHCC088','CHCC089','CHCC090','CHCC094','CHCC096','CHCC097','CHCC102','CHCC103','CHCC106','CHCC107','CHCC108','CHCC109','CHCC110','CHCC111','CHCC112','CHCC113','CHCC114','CHCC115','CHCC117','CHCC118','CHCC119','CHCC120','CHCC121','CHCC122','CHCC125','CHCC126','CHCC127','CHCC128','CHCC129','CHCC130','CHCC131','CHCC132','CHCC135','CHCC137','CHCC138','CHCC139','CHCC142','CHCC145','CHCC146','CHCC149','CHCC150','CHCC151','CHCC153','CHCC154','CHCC156','CHCC158','CHCC159','CHCC160','CHCC161','CHCC162','CHCC163','CHCC183','CHCC184','CHCC187','CHCC188','CHCC203','CHCC204','CHCC205','CHCC207','CHCC208','CHCC209','CHCC217','CHCC226','CHCC227','CHCC242','CHCC243','CHCC244','CHCC245','CHCC246','CHCC247','CHCC248','CHCC249','CHCC251','CHCC253','CHCC254']]

In [8]:
pmpm_diabetic.shape  

(34584, 128)

In [9]:
pmpm_diabetic.set_index('new_euid',inplace=True) 

In [10]:
pmpm_diabetic=pmpm_diabetic.loc[:,(pmpm_diabetic!=0).any(axis=0)] #removing 0 columns 

In [384]:
# pmpm_diabetic=pmpm_diabetic[(pmpm_diabetic.T!=0).any()] #removing 0 rows
# #(Group in itself) (Removed so that 0 dont throw error in Hclust)

In [11]:
pmpm_diabetic.shape

(34584, 75)

In [252]:
# pmpm_diabetic=pmpm_diabetic.loc[:,(pmpm_diabetic.sum()>=50)]  #dropping codes which are not there for greater than 49 patients

In [12]:
pmpm_diabetic.shape 

(34584, 75)

In [13]:
codes_stats=pd.DataFrame(pmpm_diabetic.sum()) 

In [14]:
codes_stats.head() 

,0
CHCC001,113
CHCC002,824
CHCC003,94
CHCC004,27
CHCC006,66


In [15]:
codes_stats['population_percentage']= (codes_stats[0]*1.0/34584)*100.0 

In [16]:
codes_stats.sort(['population_percentage'],ascending=False,inplace=True) 

C:\Users\IBM_ADMIN\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [17]:
codes_stats.shape 

(75, 2)

In [18]:
codes_stats.head()  

,0,population_percentage
CHCC142,3822,11.051353
CHCC130,3297,9.533310
CHCC012,1833,5.300139
CHCC088,1833,5.300139
CHCC217,1081,3.125723


In [19]:
codes_stats.columns=['Total_Patients','population_percentage'] #renaming columns

In [20]:
codes_stats.head()  

,Total_Patients,population_percentage
CHCC142,3822,11.051353
CHCC130,3297,9.533310
CHCC012,1833,5.300139
CHCC088,1833,5.300139
CHCC217,1081,3.125723


In [21]:
codes_stats.index.rename('NAME',inplace=True)   

In [22]:
codes_stats.head()  

,Total_Patients,population_percentage
NAME,,
CHCC142,3822,11.051353
CHCC130,3297,9.533310
CHCC012,1833,5.300139
CHCC088,1833,5.300139
CHCC217,1081,3.125723


## Adding Description of codes. 

In [23]:
codes_desc=pd.read_csv("C:/Projects/CDPHP/Data Dictionary/codes_description.csv")  

In [24]:
codes_desc.head() 

,NAME,TYPE,LENGTH,VARNUM,LABEL,FORMAT,FORMATL,FORMATD,Unnamed: 8
0,agecat1,1,8,59,Age Group: Policy,AGE1CAT,0,0,NaN
1,agecat2,1,8,60,Age Group: Underwriting,AGE2CAT,0,0,NaN
2,ALLOW_AMT,1,8,87,Total-Allowed,DOLLAR,20,2,NaN
3,BENEFIT_AMT,1,8,90,Total-Benefit,DOLLAR,20,2,NaN
4,CC_alzheimer,1,8,341,Alzheimers,NaN,0,0,NaN


In [25]:
codes_desc=codes_desc[['NAME','LABEL']]

In [26]:
codes_desc.head() 

,NAME,LABEL
0,agecat1,Age Group: Policy
1,agecat2,Age Group: Underwriting
2,ALLOW_AMT,Total-Allowed
3,BENEFIT_AMT,Total-Benefit
4,CC_alzheimer,Alzheimers


In [27]:
codes_stats.reset_index(inplace=True) 


In [29]:
codes_stats[:5]

,NAME,Total_Patients,population_percentage
0,CHCC142,3822,11.051353
1,CHCC130,3297,9.533310
2,CHCC012,1833,5.300139
3,CHCC088,1833,5.300139
4,CHCC217,1081,3.125723


In [30]:
codes_stats=pd.merge(codes_stats,codes_desc,left_on=['NAME'],right_on=['NAME'], how="inner") 

In [58]:
codes_stats[:100]

,NAME,Total_Patients,population_percentage,LABEL
0,CHCC142,3822,11.051353,Specified Heart Arrhythmias
1,CHCC130,3297,9.533310,Congestive Heart Failure
2,CHCC012,1833,5.300139,"Breast (Age 50+) and Prostate Cancer, Benign/U..."
3,CHCC088,1833,5.300139,Major Depressive and Bipolar Disorders
4,CHCC217,1081,3.125723,"Chronic Ulcer of Skin, Except Pressure"
5,CHCC047,933,2.697779,Acute Pancreatitis/Other Pancreatic Disorders ...
6,CHCC056,829,2.397062,Rheumatoid Arthritis and Specified Autoimmune ...
7,CHCC002,824,2.382605,"Septicemia, Sepsis, Systemic Inflammatory Resp..."
8,CHCC132,815,2.356581,Unstable Angina and Other Acute Ischemic Heart...
9,CHCC156,800,2.313208,Pulmonary Embolism and Deep Vein Thrombosis


In [32]:
codes_stats.shape 

(75, 4)

In [33]:
pmpm_diabetic.head() 

,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,CHCC012,...,CHCC162,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254
new_euid,,,,,,,,,,,,,,,,,,,,,
718167.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718201.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718203.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718212.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
718219.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [588]:
pmpm_diabetic.shape


(34584, 75)

# Aggregating duplicate rows in pmpm_diabetic

In [85]:
pmpm_diabetic_ddup=pd.DataFrame(pmpm_diabetic.groupby(pmpm_diabetic.columns.tolist(),as_index=False).size())

In [86]:
pmpm_diabetic_ddup.reset_index(inplace=True) 

In [87]:
pmpm_diabetic_ddup.head() 

,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,CHCC012,...,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19941
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,31
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,60


In [88]:
pmpm_diabetic_ddup=pmpm_diabetic_ddup.rename(columns = {0:'Total'}) 

In [89]:
pmpm_diabetic_ddup.sort('Total',ascending=False,inplace=True) #sorting based on Total column

C:\Users\IBM_ADMIN\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [90]:
pmpm_diabetic_ddup.head()   

,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,CHCC012,...,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254,Total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19941
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1089
1598,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1026
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,956
175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649


In [91]:
pmpm_diabetic_ddup.shape 

(3097, 76)

In [92]:
print(pmpm_diabetic_ddup[pmpm_diabetic_ddup['Total']>=20].shape) ## 77 Big initial clusters
print(pmpm_diabetic_ddup[pmpm_diabetic_ddup['Total']>=20].Total.sum())  
## 29K patients Representing 85% of diabetic pop. 

(77, 76)
29490


In [44]:
# pmpm_diabetic_ddup['ID']=pmpm_diabetic_ddup.index.map(str)+"-("+pmpm_diabetic_ddup.Total.map(str)+")"

In [46]:
pmpm_diabetic_ddup.set_index('ID',inplace=True)

# Hierchical clustering

In [106]:
check = pmpm_diabetic_ddup[pmpm_diabetic_ddup['Total']>=20] 
#Working with only 77 initial large clusters

In [116]:
check.ix[:,0:len(check.columns)-1].head()

,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,CHCC012,...,CHCC162,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1598,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
data_dist =pdist(check.ix[:,0:len(check.columns)-1],'jaccard') 
Z=linkage(data_dist) 

plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('Patient Group')
plt.ylabel(' Jaccard distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=100,  # show only the last p merged clusters
    show_leaf_counts=True,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    labels=list(check.index),
    show_contracted=True,  # to get a distribution impression in truncated branches
)
plt.show() 

## Retrieving the clusters

In [118]:
max_d=0.75  
clusters =fcluster(Z,max_d,criterion='distance')  
clusters  

array([ 2,  1,  1,  1,  1,  1,  1,  1,  1,  9,  1, 10,  1,  1,  1,  1, 11,
        1, 12, 13,  1, 14, 15,  1,  1, 16, 17, 18,  1, 19, 20,  1, 21, 22,
        1,  1,  1, 23,  7,  1,  1,  1, 24, 25,  1,  1,  1,  1, 26,  1,  1,
        8,  1,  1, 27,  1,  1,  5,  1,  4,  1,  1,  1,  1,  1,  1,  6,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  3])

In [119]:
check.head() 

,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,CHCC012,...,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254,Total
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,19941
75,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1089
1598,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1026
572,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,956
175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,649


In [120]:
clusters=pd.DataFrame(clusters)
clusters=clusters.rename(columns = {0:'clusters_id'})

In [121]:
check=pd.concat([check.reset_index(),clusters], axis=1)

In [122]:
check.head()

,index,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,CHCC011,...,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254,Total,clusters_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,19941,2
1,75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1089,1
2,1598,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1026,1
3,572,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,956,1
4,175,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,649,1


In [ ]:
cols = check.columns.tolist()  
cols = cols[-2:] + cols[:-2] 
check=check[cols]
check.head()

check.set_index('index',inplace=True)

In [138]:
check.set_index('index',inplace=True)

In [139]:
check.head()

,Total,clusters_id,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,...,CHCC162,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254
index,,,,,,,,,,,,,,,,,,,,,
0,19941,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75,1089,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1598,1026,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
572,956,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175,649,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


 # Analyzing cluster1       

In [140]:
cluster1=check[check['clusters_id']==1]

In [157]:
cluster1.head()

,Total,clusters_id,CHCC001,CHCC002,CHCC003,CHCC004,CHCC006,CHCC008,CHCC009,CHCC010,...,CHCC162,CHCC163,CHCC183,CHCC184,CHCC217,CHCC226,CHCC227,CHCC251,CHCC253,CHCC254
index,,,,,,,,,,,,,,,,,,,,,
75,1089,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1598,1026,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
572,956,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175,649,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
213,458,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
cluster1

7654L